# Merge permno_cusip_ticker_permco and crsp_comp_bdx to get gvkey in it

In [70]:
import pandas as pd

In [71]:
permno_ticker_permco = pd.read_csv("/media/francia/hdd_1/research_hub/csr_project/Green_patent_dataset/original_dataset/PERMNO_CUSIP_TICKER_PERMCO.csv")
permco_gvkey = pd.read_csv("/media/francia/hdd_1/research_hub/csr_project/Green_patent_dataset/original_dataset/CRSP_COMP_BDX_GVKEY.csv")

In [72]:
permno_ticker_permco['EXCHANGE'] = permno_ticker_permco['TICKER_EXCHANGE'].str.split('-', expand=True)[1]
permno_ticker_permco = permno_ticker_permco[['PERMNO', 'PERMCO', 'TICKER', 'EXCHANGE']]

In [73]:
permco_gvkey = permco_gvkey[['PERMCO', 'GVKEY']]

In [74]:
permno_ticker_gvkey = pd.merge(permno_ticker_permco, permco_gvkey, on='PERMCO', how='left')
permno_ticker_gvkey = permno_ticker_gvkey[['PERMNO', 'TICKER', 'EXCHANGE', 'GVKEY']]
permno_ticker_gvkey.dropna(inplace=True)
permno_ticker_gvkey.drop_duplicates(inplace=True)
permno_ticker_gvkey['GVKEY'] = permno_ticker_gvkey['GVKEY'].astype(int)

In [75]:
permno_ticker_gvkey = permno_ticker_gvkey[permno_ticker_gvkey['EXCHANGE'].isin(['NAS', 'NYS'])]
permno_ticker_gvkey['EXCHANGE'] = permno_ticker_gvkey['EXCHANGE'].replace({
    'NAS': 'NASDAQ',
    'NYS': 'NYSE'
})
permno_ticker_gvkey.to_csv('/media/francia/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/permno_ticker_gvkey.csv', index=False)

## Merge green patent count and forward citation

In [1]:
import pandas as pd

In [3]:
forward = pd.read_csv("../Green_patent_dataset/original_dataset/total_5yr_forward_citations_v2_correct.csv")
count = pd.read_csv("../Green_patent_dataset/original_dataset/green_patents_count.csv")

In [4]:
print("forward.columns: ",forward.columns)
print("count.columns: ",count.columns)

forward.columns:  Index(['gvkey_numeric', 'filing_year', 'total_5yr_forward_citations',
       'within_5_years'],
      dtype='object')
count.columns:  Index(['gvkey_numeric', 'filing_year', 'patents_count'], dtype='object')


In [6]:
forward_count = pd.merge(forward, count, how='left', on=['gvkey_numeric', 'filing_year'])
forward_count.to_csv('../Green_patent_dataset/merged_dataset/total_5yr_forward_citations_current_count_v2.csv', index=False)

## Merge to get ticker for permno in value

In [ ]:
import pandas as pd

In [ ]:
permno_ticker_gvkey = pd.read_csv('/media/francia/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/permno_ticker_gvkey.csv')
permno = pd.read_csv("/media/francia/hdd_1/research_hub/csr_project/Green_patent_dataset/original_dataset/green_patents_values_filing.csv")
permno = permno['permno'].unique()
permno = pd.DataFrame(permno, columns=['PERMNO'])
company = pd.read_csv("/media/francia/hdd_1/research_hub/csr_project/CSR Reporting/Dataset/company_name_all_ticker.csv")

In [ ]:
print(permno.shape)

(2660, 1)


In [ ]:
permno_ticker = permno.merge(permno_ticker_gvkey, on='PERMNO', how='left')
permno_ticker.dropna(inplace=True)
permno_ticker

,PERMNO,TICKER,EXCHANGE,GVKEY
4,10032,PLXS,NASDAQ,12945.0
7,10078,SUNW,NASDAQ,12136.0
8,10078,JAVA,NASDAQ,12136.0
11,10104,ORCL,NYSE,12142.0
12,10107,MSFT,NASDAQ,12141.0
...,...,...,...,...
3179,93345,CDXS,NASDAQ,179841.0
3180,93354,MITL,NASDAQ,174531.0
3181,93387,CODE,NYSE,163772.0
3182,93401,TNAV,NASDAQ,184101.0


In [ ]:
# 建立有效 ticker set
valid_tickers = set(company['ticker'])

# Step 1: 先處理重複的 PERMNO
def select_valid_ticker(group):
    if len(group) == 1:
        return group
    else:
        valid = group[group['TICKER'].isin(valid_tickers)]
        if len(valid) == 1:
            return valid
        else:
            return group  # 無法唯一的留給後續人工挑選

permno_ticker_selected = permno_ticker.groupby('PERMNO', group_keys=False).apply(select_valid_ticker)

# Step 2: 把已經唯一的資料挑出來
# 先找哪些 PERMNO 是重複的
duplicated_permnos = permno_ticker_selected['PERMNO'][permno_ticker_selected.duplicated('PERMNO', keep=False)].unique()

# (1) 只保留沒有出現在 duplicated list 裡的 (真正唯一的)
permno_ticker_unique = permno_ticker_selected[~permno_ticker_selected['PERMNO'].isin(duplicated_permnos)]

# (2) 還有多筆重複的資料 (需要人工手挑)
permno_ticker_still_duplicated = permno_ticker_selected[permno_ticker_selected['PERMNO'].isin(duplicated_permnos)]

# Step 3: 儲存結果
permno_ticker_unique.to_csv('/media/francia/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/permno_ticker_unique.csv', index=False)
# permno_ticker_still_duplicated.to_csv('permno_ticker_still_duplicated.csv', index=False)

# Summary
print(f"總共處理後有 {len(permno_ticker_unique)} 筆唯一 PERMNO，已儲存到 permno_ticker_unique_final.csv")
print(f"仍有 {len(permno_ticker_still_duplicated['PERMNO'].unique())} 個 PERMNO 還有重複，儲存到 permno_ticker_still_duplicated.csv，供人工挑選")


總共處理後有 1067 筆唯一 PERMNO，已儲存到 permno_ticker_unique_final.csv
仍有 223 個 PERMNO 還有重複，儲存到 permno_ticker_still_duplicated.csv，供人工挑選


/tmp/ipykernel_1022002/1503063878.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  permno_ticker_selected = permno_ticker.groupby('PERMNO', group_keys=False).apply(select_valid_ticker)


In [ ]:
permno_ticker_unique

,PERMNO,TICKER,EXCHANGE,GVKEY
4,10032,PLXS,NASDAQ,12945.0
11,10104,ORCL,NYSE,12142.0
12,10107,MSFT,NASDAQ,12141.0
18,10145,HON,NYSE,1300.0
21,10153,AH,NYSE,1308.0
...,...,...,...,...
3179,93345,CDXS,NASDAQ,179841.0
3180,93354,MITL,NASDAQ,174531.0
3181,93387,CODE,NYSE,163772.0
3182,93401,TNAV,NASDAQ,184101.0


In [ ]:
permno_ticker_still_duplicated

,PERMNO,TICKER,EXCHANGE,GVKEY
7,10078,SUNW,NASDAQ,12136.0
8,10078,JAVA,NASDAQ,12136.0
14,10143,ONCS,NASDAQ,12180.0
15,10143,OSIP,NASDAQ,12180.0
19,10147,EMCS,NYSE,12053.0
...,...,...,...,...
3097,91832,MDGL,NASDAQ,27716.0
3098,91836,HSTX,NASDAQ,176572.0
3099,91836,AVNW,NASDAQ,176572.0
3139,92453,SYMX,NASDAQ,160592.0


In [ ]:
# # 建立有效的 ticker set
# valid_tickers = set(company['ticker'])

# # 篩選出 permno_ticker 中 TICKER 在 company 資料中的資料
# valid_permno_ticker = permno_ticker[permno_ticker['TICKER'].isin(valid_tickers)]

# # 儲存結果
# # valid_permno_ticker.to_csv('valid_permno_ticker.csv', index=False)

# # 輸出篩選後的資料筆數
# print(f"共篩選出 {len(valid_permno_ticker)} 筆資料，已儲存到 valid_permno_ticker.csv")

共篩選出 601 筆資料，已儲存到 valid_permno_ticker.csv


### merge value to get ticker

In [ ]:
import pandas as pd

In [ ]:
gp_value = pd.read_csv("/media/francia/hdd_1/research_hub/csr_project/Green_patent_dataset/original_dataset/green_patents_values_filing.csv")
permno_ticker = pd.read_csv("/media/francia/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/permno_ticker_unique.csv")

In [ ]:
gp_value

,permno,year,total_values_nominal,patents_count_nominal,total_values_real,patents_count_real,value_per_patent_nominal,value_per_patent_real
0,10001,1993.0,0.190015,1,0.123884,1,0.190015,0.123884
1,10001,2002.0,0.391408,1,0.194113,1,0.391408,0.194113
2,10006,1929.0,0.019827,1,0.120193,1,0.019827,0.120193
3,10006,1932.0,0.104737,2,0.750615,2,0.052368,0.375307
4,10006,1937.0,0.053046,1,0.369803,1,0.053046,0.369803
...,...,...,...,...,...,...,...,...
17796,93436,2015.0,1115.396654,4,452.246280,4,278.849163,113.061570
17797,93436,2016.0,573.500674,1,217.818401,1,573.500674,217.818401
17798,93436,2019.0,6813.554617,2,2167.926984,2,3406.777308,1083.963492
17799,93436,2021.0,4826.775018,2,1535.776316,2,2413.387509,767.888158


In [ ]:
permno_ticker

,PERMNO,TICKER,EXCHANGE,GVKEY
0,10032,PLXS,NASDAQ,12945
1,10104,ORCL,NYSE,12142
2,10107,MSFT,NASDAQ,12141
3,10145,HON,NYSE,1300
4,10153,AH,NYSE,1308
...,...,...,...,...
1062,93345,CDXS,NASDAQ,179841
1063,93354,MITL,NASDAQ,174531
1064,93387,CODE,NYSE,163772
1065,93401,TNAV,NASDAQ,184101


In [ ]:
gp_value_ticker = gp_value.merge(permno_ticker[['PERMNO','TICKER','EXCHANGE']], left_on='permno', right_on='PERMNO', how='left')
gp_value_ticker.dropna(inplace=True)
gp_value_ticker.drop_duplicates(inplace=True)
gp_value_ticker.drop(columns=['PERMNO'], inplace=True)
gp_value_ticker.rename(columns={'TICKER': 'ticker', 'EXCHANGE': 'exchange'}, inplace=True)
gp_value_ticker['year'] = gp_value_ticker['year'].astype(int)

In [ ]:
# 檢查重複的 permno + year 組合
duplicates = gp_value_ticker[gp_value_ticker.duplicated(subset=['permno', 'year'], keep=False)]

# 顯示結果
if not duplicates.empty:
    print(f"共有 {len(duplicates)} 筆資料在 permno + year 上有重複組合。")
    print(duplicates[['permno', 'year']].drop_duplicates())
else:
    print("沒有重複的 permno + year 組合。")

沒有重複的 permno + year 組合。


In [ ]:
gp_value_ticker.to_csv("/media/francia/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/green_patents_values_ticker.csv", index=False)

## Merge to get ticker for gvkey in forward count

In [7]:
import pandas as pd

In [ ]:
permno_ticker_gvkey = pd.read_csv('/media/francia/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/permno_ticker_gvkey.csv')
gvkey = pd.read_csv("/media/francia/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/total_5yr_forward_citations_current_count_v2.csv")
gvkey = gvkey['gvkey_numeric'].unique()
gvkey = pd.DataFrame(gvkey, columns=['GVKEY'])
company = pd.read_csv("/media/francia/hdd_1/research_hub/csr_project/CSR Reporting/Dataset/company_name_all_ticker.csv")

In [9]:
print(gvkey.shape)

(12417, 1)


In [10]:
gvkey_ticker = gvkey.merge(permno_ticker_gvkey, on='GVKEY', how='left')
gvkey_ticker.dropna(inplace=True)
gvkey_ticker

,GVKEY,PERMNO,TICKER,EXCHANGE
0,8972.0,24942.0,RTN,NYSE
1,8972.0,85658.0,RTN,NYSE
3,1690.0,14593.0,AAPL,NASDAQ
4,2285.0,19561.0,BA,NYSE
5,5608.0,61567.0,HEXC,NYSE
...,...,...,...,...
14079,26305.0,80037.0,SLVN,NASDAQ
14080,26305.0,80037.0,LAUR,NASDAQ
14082,162449.0,91500.0,EXLS,NASDAQ
14093,23717.0,76577.0,SMTK,NASDAQ


In [11]:
# 建立有效 ticker set
valid_tickers = set(company['ticker'])

# Step 1: 先處理重複的 PERMNO
def select_valid_ticker(group):
    if len(group) == 1:
        return group
    else:
        valid = group[group['TICKER'].isin(valid_tickers)]
        if len(valid) == 1:
            return valid
        else:
            return group  # 無法唯一的留給後續人工挑選

gvkey_ticker_selected = gvkey_ticker.groupby('GVKEY', group_keys=False).apply(select_valid_ticker)

# Step 2: 把已經唯一的資料挑出來
# 先找哪些 GVKEY 是重複的
duplicated_permnos = gvkey_ticker_selected['GVKEY'][gvkey_ticker_selected.duplicated('GVKEY', keep=False)].unique()

# (1) 只保留沒有出現在 duplicated list 裡的 (真正唯一的)
gvkey_ticker_unique = gvkey_ticker_selected[~gvkey_ticker_selected['GVKEY'].isin(duplicated_permnos)]

# (2) 還有多筆重複的資料 (需要人工手挑)
gvkey_ticker_still_duplicated = gvkey_ticker_selected[gvkey_ticker_selected['GVKEY'].isin(duplicated_permnos)]

# Step 3: 儲存結果
# gvkey_ticker_unique.to_csv('gvkey_ticker_unique_final.csv', index=False)
# gvkey_ticker_still_duplicated.to_csv('gvkey_ticker_still_duplicated.csv', index=False)

# Summary
print(f"總共處理後有 {len(gvkey_ticker_unique)} 筆唯一 GVKEY，已儲存到 gvkey_ticker_unique_final.csv")
print(f"仍有 {len(gvkey_ticker_still_duplicated['GVKEY'].unique())} 個 GVKEY 還有重複，儲存到 gvkey_ticker_still_duplicated.csv，供人工挑選")


總共處理後有 2575 筆唯一 GVKEY，已儲存到 gvkey_ticker_unique_final.csv
仍有 743 個 GVKEY 還有重複，儲存到 gvkey_ticker_still_duplicated.csv，供人工挑選


/tmp/ipykernel_1215028/2380978070.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  gvkey_ticker_selected = gvkey_ticker.groupby('GVKEY', group_keys=False).apply(select_valid_ticker)


In [12]:
gvkey_ticker_unique.to_csv('/media/francia/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/gvkey_ticker_unique.csv', index=False)

In [13]:
permno_ticker_still_duplicated

NameError: name 'permno_ticker_still_duplicated' is not defined

### merge forward count to get ticker

In [14]:
import pandas as pd

In [15]:
forward_count = pd.read_csv("/media/francia/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/total_5yr_forward_citations_current_count_v2.csv")
gvkey_ticker = pd.read_csv("/media/francia/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/gvkey_ticker_unique.csv")

In [16]:
forward_count

,gvkey_numeric,filing_year,total_5yr_forward_citations,within_5_years,patents_count
0,8972.0,2000,0.0,0.0,0.0
1,8972.0,2001,0.0,NaN,0.0
2,8972.0,2002,0.0,NaN,0.0
3,8972.0,2003,0.0,0.0,0.0
4,8972.0,2004,0.0,0.0,0.0
...,...,...,...,...,...
248335,288914.0,2015,1.0,1.0,NaN
248336,288914.0,2016,1.0,1.0,NaN
248337,288914.0,2017,0.0,NaN,NaN
248338,288914.0,2018,0.0,NaN,NaN


In [17]:
gvkey_ticker

,GVKEY,PERMNO,TICKER,EXCHANGE
0,1004.0,54594.0,AIR,NYSE
1,1013.0,50906.0,ADCT,NASDAQ
2,1045.0,21020.0,AAL,NASDAQ
3,1050.0,11499.0,CECO,NASDAQ
4,1078.0,20482.0,ABT,NYSE
...,...,...,...,...
2570,293919.0,15794.0,SBBP,NASDAQ
2571,311524.0,13861.0,TAM,NYSE
2572,315318.0,14406.0,PAH,NYSE
2573,316056.0,14297.0,ALLE,NYSE


In [18]:
forward_count_ticker = forward_count.merge(gvkey_ticker, left_on='gvkey_numeric', right_on='GVKEY', how='left')
forward_count_ticker.dropna(inplace=True)
forward_count_ticker.drop_duplicates(inplace=True)
forward_count_ticker.drop(columns=['GVKEY'], inplace=True)
forward_count_ticker.rename(columns={'TICKER': 'ticker', 'EXCHANGE': 'exchange', 'filing_year': 'year'}, inplace=True)
forward_count_ticker['year'] = forward_count_ticker['year'].astype(int)
forward_count_ticker.drop(columns=['PERMNO'], inplace=True)

In [19]:
# 檢查重複的 permno + year 組合
duplicates = forward_count_ticker[forward_count_ticker.duplicated(subset=['gvkey_numeric', 'year'], keep=False)]

# 顯示結果
if not duplicates.empty:
    print(f"共有 {len(duplicates)} 筆資料在 permno + year 上有重複組合。")
    print(duplicates[['permno', 'year']].drop_duplicates())
else:
    print("沒有重複的 permno + year 組合。")

沒有重複的 permno + year 組合。


In [20]:
forward_count_ticker.to_csv("/media/francia/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/total_5yr_forward_citations_current_count_ticker_v2.csv", index=False)

# Merge forward count and value

In [21]:
import pandas as pd
import numpy as np

In [22]:
forward_count_ticker = pd.read_csv("/media/francia/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/total_5yr_forward_citations_current_count_ticker_v2.csv")
value = pd.read_csv("/media/francia/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/green_patents_values_ticker.csv")

In [23]:
forward_count_ticker.head()

,gvkey_numeric,year,total_5yr_forward_citations,within_5_years,patents_count,ticker,exchange
0,1690.0,2000,0.0,0.0,0.0,AAPL,NASDAQ
1,1690.0,2001,0.0,0.0,0.0,AAPL,NASDAQ
2,1690.0,2002,0.0,0.0,0.0,AAPL,NASDAQ
3,1690.0,2003,0.0,0.0,0.0,AAPL,NASDAQ
4,1690.0,2004,0.0,0.0,1.0,AAPL,NASDAQ


In [24]:
value.head()

,permno,year,total_values_nominal,patents_count_nominal,total_values_real,patents_count_real,value_per_patent_nominal,value_per_patent_real,ticker,exchange
0,10032,1997,4.634948,1,2.454536,1,4.634948,2.454536,PLXS,NASDAQ
1,10104,1997,3536.844015,2,2030.748163,2,1768.422007,1015.374081,ORCL,NYSE
2,10104,1999,606.861224,1,327.416559,1,606.861224,327.416559,ORCL,NYSE
3,10104,2001,121.758702,1,56.527434,1,121.758702,56.527434,ORCL,NYSE
4,10104,2002,131.071437,2,60.572058,2,65.535718,30.286029,ORCL,NYSE


In [25]:
value['ticker'] = value['ticker'].str.upper()
forward_count_ticker['ticker'] = forward_count_ticker['ticker'].str.upper()

In [26]:
import pandas as pd

# 假設你已經有 forward_count_ticker 和 value

merged = pd.merge(
    forward_count_ticker,
    value,
    how='outer',
    on=['ticker', 'year']
)

merged.head()


,gvkey_numeric,year,total_5yr_forward_citations,within_5_years,patents_count,ticker,exchange_x,permno,total_values_nominal,patents_count_nominal,total_values_real,patents_count_real,value_per_patent_nominal,value_per_patent_real,exchange_y
0,NaN,1997,NaN,NaN,NaN,A,NaN,87432.0,116.338665,1.0,64.259500,1.0,116.338665,64.259500,NYSE
1,NaN,1998,NaN,NaN,NaN,A,NaN,87432.0,27.710206,1.0,15.305694,1.0,27.710206,15.305694,NYSE
2,NaN,1999,NaN,NaN,NaN,A,NaN,87432.0,213.287241,4.0,118.740667,4.0,53.321810,29.685167,NYSE
3,NaN,2000,NaN,NaN,NaN,A,NaN,87432.0,54.201901,3.0,29.243259,3.0,18.067300,9.747753,NYSE
4,126554.0,2001,0.0,0.0,0.0,A,NYSE,87432.0,122.779864,11.0,63.211683,11.0,11.161806,5.746517,NYSE


In [27]:
merged.dropna(inplace=True)
merged.drop_duplicates(inplace=True)
merged.drop(columns=['gvkey_numeric', 'permno', 'exchange_y'], inplace=True)
merged.rename(columns={'exchange_x': 'exchange'}, inplace=True)
merged

,year,total_5yr_forward_citations,within_5_years,patents_count,ticker,exchange,total_values_nominal,patents_count_nominal,total_values_real,patents_count_real,value_per_patent_nominal,value_per_patent_real
4,2001,0.0,0.0,0.0,A,NYSE,122.779864,11.0,63.211683,11.0,11.161806,5.746517
5,2002,0.0,0.0,0.0,A,NYSE,37.774669,5.0,19.117970,5.0,7.554934,3.823594
6,2003,0.0,0.0,0.0,A,NYSE,315.327461,21.0,148.880364,21.0,15.015593,7.089541
7,2004,0.0,0.0,1.0,A,NYSE,32.921879,4.0,15.518458,4.0,8.230470,3.879614
9,2006,0.0,0.0,2.0,A,NYSE,234.731334,6.0,101.729252,6.0,39.121889,16.954875
...,...,...,...,...,...,...,...,...,...,...,...,...
16720,2009,32.0,4.0,2.0,ZTS,NYSE,147.311373,2.0,61.484688,2.0,73.655687,30.742344
16722,2012,40.0,7.0,0.0,ZTS,NYSE,128.074195,1.0,51.774584,1.0,128.074195,51.774584
16723,2013,62.0,12.0,1.0,ZTS,NYSE,59.486840,1.0,23.551117,1.0,59.486840,23.551117
16724,2014,34.0,7.0,1.0,ZTS,NYSE,82.642901,1.0,32.105445,1.0,82.642901,32.105445


In [28]:
merged.to_csv("/media/francia/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/forward_count_value_ticker_all_y_v2.csv", index=False)